## (2) /header/received/0/src에서 날짜+시간 정보 파싱

eml header 중 
/header/received/0/src에는 다음과 같이 2가지(일단 확인한 것은 2가지) 형태로 정보가 들어있다.
- 형식 1) from [ip] at [날짜 + 시간] by [mail.com] with [esmtp credishield]
- 형식 2) from [imss.checkbox.kr] by [imsva] with [esmtp] id [id] for [메일주소]; [날짜+시간]

형식 2의 경우 모든 내용이 /header/received/0/from, /header/received/0/by, /header/received/0/with, /header/received/0/id, /header/received/0/for, /header/received/0/date에 파싱되어 저장되어 있음(이제까지 확인한 것에 따르면, 정확하게 확인할 필요 있지만, 아직 안함)
<br>형식 1의 경우 다른 내용은 형식 2와 같이 파싱되어 각 열(/header/received/0/from, /header/received/0/by, ...)에 저장되어 있지만, 날짜는 제대로 저장되지 않음 1970-01-01 00:00:00+00:00 으로 저장되어 있음
<br>
<br>따라서 /header/received/0/src 열의 내용이 형식1과 같이 된 경우에 관해서만, 그 내용 중 날짜만 처리함 /header/received/0/src 안에서 날자+시간 정보를 빼내어 /header/received/0/src_at 열에 저장
<br>만약 /header/received/0/src 열의 내용이 형식2라면 /header/received/0/src_at에는 Naf값이 저장됨
<br>이에 해당하는 행들은 마지막에 /header/received/0/date 열의 내용을 /header/received/0/src_at 열에 저장

In [1]:
import pandas as pd
import datetime

In [2]:
eml_df = pd.read_csv('../../parsed_eml_1.csv') # 이전에 파싱하여 만든 csv 파일 읽어오기

In [3]:
# '/header/received/0'로 시작하는 열이름 저장
# '/header/received/0/src는 check_src_columns에 저장
# '/header/received/0/from', /header/received/0/date', .. 은 check_columns에 저장
check_columns = []
check_src_columns = []
for c in eml_df.columns:
    if '/header/received/0' in c:
        if '/src' in c:
            check_src_columns.append(c)    
        else:
            check_columns.append(c)


In [4]:
# 각 열에 해당하는 내용을 eml_df에서 뽑아서 새로운 check_df, check_src_df 만듬
check_df = pd.DataFrame()
for col in check_columns:
    check_df = pd.concat([check_df, eml_df[col]], axis=1)
    
check_src_df = pd.DataFrame()
for col in check_src_columns:
    check_src_df = pd.concat([check_src_df, eml_df[col]], axis=1)

In [5]:
# 처리할 열(/header/received/0/src)과 그 열에 대한 값이 저장되어 있는 check_src_df를 인자로 받음 
# /header/received/0/src의 내용이 형식 1과 같은 경우 
# 'from ', ' at ', ' by ', ' with ' 문자열을 기준으로 split
# split한 결과를 각각 열에 추가하여 all_df DataFrame을 반환
# 만약 /header/received/0/src 내용이 형식 2라면, 중간에 split되지 않아,
# /header/received/0/src_at열에는 Naf값이 저장, from은 저장될 것.(아직 깔끔하게 처리 못함. 형식 1에 대해서만 처리함)
def split_src(check_src_columns, check_src_df):
    all_df = pd.DataFrame()
    for i, col in enumerate(check_src_columns):
        check_df = pd.DataFrame()
        if check_src_df[col].isnull().sum() == len(check_src_df):
            continue
        temp = check_src_df[col].str.split('from ', expand=True)
        if len(temp.columns) >= 2:
            split_from = temp[1].str.split(' at ', expand=True)
            check_df=pd.concat([check_df, split_from[0]], axis=1)
            check_df.columns = [col+'_from']
            if len(split_from.columns) >= 2:
                split_at = split_from[1].str.split(' by ', expand=True)
                check_df=pd.concat([check_df, split_at[0]], axis=1)
                check_df.columns = [col+'_from', col+'_at']
                if len(split_at.columns) >= 2:
                    split_by = split_at[1].str.split(' with ', expand=True)
                    check_df=pd.concat([check_df, split_by[0]], axis=1)
                    check_df.columns = [col+'_from', col+'_at', col+'_by']
                    if len(split_by.columns) >= 2:
                        check_df=pd.concat([check_df, split_by[1]], axis=1)
                        check_df.columns = [col+'_from', col+'_at', col+'_by', col+'_with']

            print('**', check_df.columns)
            all_df=pd.concat([all_df, check_df])
 
    return all_df

In [15]:
# /header/received/0/src 를 split한 결과를 splited_df에 저장
splited_df = split_src(check_src_columns, check_src_df)

** Index(['/header/received/0/src_from', '/header/received/0/src_at',
       '/header/received/0/src_by', '/header/received/0/src_with'],
      dtype='object')


In [16]:
splited_df

,/header/received/0/src_from,/header/received/0/src_at,/header/received/0/src_by,/header/received/0/src_with
0,58.248.209.14(58.248.209.14),"thu, 29 aug 2019 20:22:27 +0900",mail.com,esmtp credishield
1,122.97.133.171(122.97.133.171),"mon, 06 apr 2020 05:06:38 +0900",mail.com,esmtp credishield
2,219.139.240.195(219.139.240.195),"fri, 05 jul 2019 15:27:12 +0900",mail.com,esmtp credishield
3,92.118.114.40(92.118.114.40),"thu, 11 jun 2020 01:28:26 +0900",mail.com,esmtp credishield
4,200.119.125.194(200.119.125.194),"sat, 16 may 2020 11:40:25 +0900",mail.com,esmtp credishield
...,...,...,...,...
150,196.216.220.204(196.216.220.204),"mon, 06 apr 2020 15:20:31 +0900",mail.com,esmtp credishield
151,180.166.200.170(180.166.200.170),"tue, 14 apr 2020 03:00:47 +0900",mail.com,esmtp credishield
152,180.167.196.58(180.167.196.58),"mon, 06 apr 2020 07:11:17 +0900",mail.com,esmtp credishield
153,180.166.200.170(180.166.200.170),"tue, 14 apr 2020 00:12:15 +0900",mail.com,esmtp credishield


In [7]:
# 날짜+시간에 관련된 내용을 뽑아 date_time_str에 저장
#date_time_str = splited_df['/header/received/0/src_at']

In [8]:
# to_datetime을 이용해 날짜 형식으로 변환하여 다시 '/header/received/0/src_at'열에 저장
#splited_df['/header/received/0/src_at'] = pd.to_datetime(date_time_str, utc=True)

In [17]:
# 이전의 열들 뒤에 추가하여 새로운 DataFrame 정의
splited_and_concat_df = pd.concat([eml_df, splited_df['/header/received/0/src_at']], axis=1)

In [18]:
# 형식 2의 경우 '/header/received/0/src_at'에 Naf(null)값이 들어있기 때문에 이를 처리
# '/header/received/0/src_at' 값이 null인 행들은 '/header/received/0/date' 값을 '/header/received/0/src_at'에 저장시킴
is_not_date = splited_and_concat_df['/header/received/0/src_at'].isnull()
splited_and_concat_df.loc[is_not_date,'/header/received/0/src_at'] = splited_and_concat_df[is_not_date]['/header/received/0/date']


In [19]:
# EDA에 사용할 csv 파일 생성
splited_and_concat_df.to_csv('../../parsed_eml_2.csv', encoding='utf-8-sig')